<a href="https://colab.research.google.com/github/mathgds/introducao_a_ciencia_de_dados/blob/main/04_bias_variance_tradeoff_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entendendo o Compromisso entre Viés e Variância


## 1. Introdução
Neste notebook, exploraremos os conceitos de viés e variância, que são cruciais para entender o desempenho dos modelos de aprendizado de máquina. Demonstrararemos esses conceitos usando dados sintéticos, modelos de regressão linear simples e validação cruzada.

## 2. Importando bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures from

# sklearn.preprocessing: Importa a partir do módulo preprocessing da biblioteca sklearn (scikit-learn).
#                        O módulo preprocessing contém várias funções e classes para a transformação e pré-processamento de dados.
#                        PolynomialFeatures é uma classe que gera características polinomiais a partir de características de entrada.
#                        Essa transformação é útil para ajustar modelos de regressão polinomial.

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from mlxtend.evaluate import bias_variance_decomp
# from mlxtend.evaluate: Importa a partir do módulo evaluate da biblioteca mlxtend. O mlxtend (Machine Learning Extensions)
#                        é uma biblioteca que fornece uma variedade de ferramentas e extensões para aprendizado de máquina que não estão disponíveis diretamente no scikit-learn.

# bias_variance_decomp: bias_variance_decomp é uma função que calcula a decomposição do viés e da variância de um modelo, ajudando a analisar o compromisso entre viés e variância.

## 3. Criando dados sintéticos

In [ ]:
# Função para gerar dados sintéticos
def generate_synthetic_data(n_samples=100, noise=1.0, random_seed=42):
# Parâmetros:
# n_samples (100): Número de amostras a serem geradas.
# noise (1.0): Nível de ruído a ser adicionado aos dados.
# random_seed (42): Semente para garantir reprodutibilidade dos dados gerados.

    np.random.seed(random_seed)
     # Define a semente aleatória para garantir que os resultados sejam reprodutíveis.

    X = np.linspace(0, 100, n_samples).reshape(-1, 1) # Criação das Amostras de Entrada
    # np.linspace(0, 100, n_samples):
    # Gera n_samples valores igualmente espaçados entre 0 e 100.
    # reshape(-1, 1):
    # Converte o array de uma dimensão em um array bidimensional com uma única coluna. Isso é necessário para que X tenha a forma correta para treinamento de modelos.

    true_function = -0.0001 * X**3 + 0.01 * X**2 + 0.1 * X + 1
    # Define a função verdadeira que gera os valores de y sem ruído. Aqui, a função é um polinômio cúbico (terceiro grau) com coeficientes específicos.

    y = true_function + np.random.normal(scale=noise, size=X.shape)
    # np.random.normal(scale=noise, size=X.shape):
    # Adiciona ruído gaussiano aos valores da função verdadeira. O ruído é gerado com uma distribuição normal com desvio padrão especificado pelo parâmetro noise.
    # y: Os valores finais da saída, que são a soma da função verdadeira e do ruído.

    return X, y, true_function
    # Retorna três valores:
    # X: A matriz de características de entrada.
    # y: A matriz de saídas com ruído.
    # true_function: A função verdadeira usada para gerar y, sem ruído.

# Gerando dados sintéticos
X, y, true_function = generate_synthetic_data(n_samples=100, noise=2.0)

In [ ]:
# Dividindo em dados de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# O código utiliza a função train_test_split do scikit-learn para dividir os dados em conjuntos de treinamento e teste.
# A divisão é feita de forma que 80% dos dados são usados para treinar o modelo e 20% são reservados para testar o modelo.

## 4. Modelo simples de regressão linear

In [ ]:
# Treinando o modelo
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Previsões
y_train_pred = lin_reg.predict(X_train)
y_test_pred = lin_reg.predict(X_test)

In [ ]:
# Plotando os resultados
plt.scatter(X_train, y_train, color='blue', label='Training data')
plt.scatter(X_test, y_test, color='green', label='Testing data')
plt.plot(X_train, y_train_pred, color='red', label='Linear regression')
plt.xlabel('Feature')
plt.ylabel('Target')
plt.title('Linear Regression Model')
plt.legend()
plt.show()

In [ ]:
# Calculando os erros de treinamento e de teste
train_error = mean_squared_error(y_train, y_train_pred)
# train_error: Resultado do cálculo do MSE para os dados de treinamento.
#              Representa a média dos quadrados dos erros entre os valores reais e os valores previstos para o conjunto de treinamento.

test_error = mean_squared_error(y_test, y_test_pred)
# test_error: Resultado do cálculo do MSE para os dados de teste.

print(f'Training Error: {train_error}')
print(f'Testing Error: {test_error}')

Quando você observa que o Erro Quadrático Médio (MSE) para os dados de treinamento é maior do que o MSE para os dados de teste, isso é atípico, mas pode ocorrer por várias razões. Aqui está uma explicação das possíveis causas:

1. Complexidade do Modelo e Ajuste

- Sobreajuste: Normalmente, o sobreajuste ocorre quando o modelo se sai bem com os dados de treinamento, mas mal com os dados de teste. Nesses casos, o MSE de treinamento seria tipicamente menor do que o MSE de teste. Se o MSE de treinamento é maior, isso sugere que o modelo pode não estar sobreajustado, mas pode estar subajustado ou haver outro problema.

2. Questões com os Dados

- Qualidade dos Dados de Treinamento: A qualidade e a distribuição dos dados de treinamento podem diferir dos dados de teste. Se os dados de treinamento são ruidosos ou contêm anomalias, isso pode levar a um MSE de treinamento mais alto em comparação com o MSE de teste.

- Simplicidade dos Dados de Teste: Os dados de teste podem ser mais simples ou ter menos variabilidade do que os dados de treinamento. Isso pode levar a um MSE mais baixo nos dados de teste, já que o modelo encontra mais fácil prever os dados de teste com precisão.

3. Problemas de Treinamento do Modelo

- Treinamento Insuficiente: Se o modelo não foi treinado adequadamente ou não convergiu corretamente, ele pode ter um desempenho pior nos dados de treinamento. Isso pode levar a um MSE de treinamento mais alto do que o MSE de teste. Por exemplo, se o processo de treinamento foi interrompido ou não foi permitido rodar por um número suficiente de épocas, o modelo pode não ter aprendido os padrões nos dados de treinamento bem.

4. Variações Aleatórias
- Semente Aleatória e Divisão dos Dados:
Variações aleatórias na forma como os dados foram divididos ou amostrados podem levar a resultados incomuns. Diferentes execuções do mesmo experimento podem mostrar variabilidade devido a divisões aleatórias ou geração de dados.

5. Regularização e Restrições


- Regularização:
Se um termo de regularização é aplicado e é forte, ele pode penalizar o modelo severamente por ajustar os dados de treinamento muito de perto. Isso pode levar a um MSE de treinamento mais alto em comparação com o MSE de teste.

## 5. Modelo de Regressão Polinomial

In [ ]:
# Ajustar e plotar modelos polinomiais de diferentes graus
degrees = [1, 2, 3, 15]
colors = ['orange', 'red', 'black', 'cyan']
predictions = []

for degree, color in zip(degrees, colors):    # for percorre simultaneamente duas listas:
                                              # degrees e colors, usando a função zip. Cada iteração fornece um valor de degree (grau do polinômio) e um valor de color
                                              # (cor para a visualização).

    poly_features = PolynomialFeatures(degree=degree, include_bias=False) # Esta classe do scikit-learn transforma as características de
                                                                          # entrada em um polinômio de um determinado grau.

    X_poly = poly_features.fit_transform(X)
    # Ajusta o transformador aos dados de entrada X e transforma X para incluir características polinomiais de acordo com o grau especificado.

    model = LinearRegression()
    model.fit(X_poly, y)
    y_poly_pred = model.predict(X_poly)
    predictions.append(y_poly_pred)
    plt.plot(X, y_poly_pred, label=f'Degree {degree}', color=color)

# Plotando os dados sintéticos e a função real
plt.scatter(X, y, facecolors='none', edgecolors='black', label='Data')
plt.plot(X, true_function, label='True Function', color='blue')
plt.xlabel('Feature (X)')
plt.ylabel('Target (Y)')
plt.title('Data and Polynomial Fits')
plt.legend()
plt.show()

## 6. Compromisso entre Viés e Variância

O compromisso entre viés e variância é um conceito fundamental em aprendizado de máquina. Ele descreve o compromisso entre duas fontes de erro que afetam o desempenho de um modelo:

- **Viés**: Erro devido a suposições excessivamente simplistas no algoritmo de aprendizado. Um viés alto pode fazer com que o modelo perca as relações relevantes entre as características e as saídas-alvo (subajuste).

- **Variância**: Erro devido a uma complexidade excessiva no algoritmo de aprendizado. Uma variância alta pode fazer com que o modelo ajuste o ruído aleatório nos dados de treinamento (sobreajuste).

Para visualizar o compromisso entre viés e variância, podemos plotar os erros de treinamento e teste para modelos com diferentes complexidades.

In [ ]:
# Calculando e plotando o MSE de treinamento e teste
# Inicialização das Listas
train_errors = []
test_errors = []
flexibility = []

for degree in range(1, 21): # O loop for percorre os graus de polinômio de 1 a 20. Cada iteração testa um grau diferente de polinômio.
    poly_features = PolynomialFeatures(degree=degree, include_bias=False) # Cria uma transformação polinomial para o grau atual.
    X_train_poly = poly_features.fit_transform(X_train) # Aplica a transformação polinomial aos dados de treinamento.
    X_test_poly = poly_features.fit_transform(X_test) # Aplica a transformação polinomial aos dados de teste.

    model = LinearRegression()
    model.fit(X_train_poly, y_train)

    y_train_pred = model.predict(X_train_poly)
    y_test_pred = model.predict(X_test_poly)

    train_errors.append(mean_squared_error(y_train, y_train_pred))
    test_errors.append(mean_squared_error(y_test, y_test_pred))
    flexibility.append(degree)

plt.plot(flexibility, train_errors, label='Training MSE', color='gray', marker='o')
plt.plot(flexibility, test_errors, label='Test MSE', color='red', marker='o')
plt.xlabel('Flexibility (Polynomial Degree)')
plt.ylabel('Error')
plt.title('Bias-Variance Tradeoff')
plt.xticks(flexibility)  # Mostrar valores inteiros dos graus no eixo x
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

## 7. Conclusão

Neste notebook, exploramos os conceitos de viés e variância e demonstramos como eles afetam o desempenho dos modelos de aprendizado de máquina. Treinamos modelos de regressão linear e polinomial com dados sintéticos e visualizamos o compromisso entre viés e variância.

Compreender e gerenciar o compromisso entre viés e variância é crucial para construir modelos que generalizem bem para novos dados não vistos. Ao equilibrar viés e variância, podemos alcançar um melhor desempenho e previsões mais confiáveis.